In [15]:
#@title Installing libraries { display-mode: "form" }
!pip3 install transformers 
!pip3 install tensorboard 
!pip3 install simpletransformers
!pip3 install jsonlines
!pip3 install pytelegrambotapi
!pip3 install tqdm

In [16]:
#@title Setup & Config & Imports
!mkdir /usr/local/lib/python3.6/dist-packages/tqdm-4.41.1.dist-info/METADATA
import tqdm
class _TQDM(tqdm.tqdm):
    def __init__(self, *argv, **kwargs):
        kwargs['disable'] = True
        if kwargs.get('disable_override', 'def') != 'def':
            kwargs['disable'] = kwargs['disable_override']
        super().__init__(*argv, **kwargs)
tqdm.tqdm = _TQDM
 
import pandas as pd
import numpy as np
import json
import jsonlines
import re
from collections import defaultdict, Counter
 
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.color'] = mpl.rcParams['axes.labelcolor'] = mpl.rcParams['xtick.color'] = mpl.rcParams['ytick.color'] = 'white'
 
%pylab inline
from sklearn.metrics import *
 
try:
    from simpletransformers.classification import ClassificationModel, ClassificationArgs
except (FileNotFoundError, AttributeError):
    from simpletransformers.classification import ClassificationModel, ClassificationArgs
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
 
%load_ext tensorboard
 
import string
 
RS = 179
PUNCTUATION = string.punctuation
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

mkdir: cannot create directory ‘/usr/local/lib/python3.6/dist-packages/tqdm-4.41.1.dist-info/METADATA’: No such file or directory
Populating the interactive namespace from numpy and matplotlib
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Building jupyterlab assets (build:prod:minimize)


In [17]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

Building jupyterlab assets (build:prod:minimize)


In [18]:
!rm -rf sample_data/

# Downloading data

In [19]:
!curl https://onti2020.ai-academy.ru/task/rucos.zip -o data.zip
!unzip -o data.zip

!mv RuCoS/ data/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 53.6M  100 53.6M    0     0  13.4M      0  0:00:03  0:00:03 --:--:-- 13.4M
Archive:  data.zip
  inflating: __MACOSX/._RuCoS        
  inflating: RuCoS/.DS_Store         
  inflating: __MACOSX/RuCoS/._.DS_Store  
  inflating: RuCoS/rucos_test.jsonl  
  inflating: __MACOSX/RuCoS/._rucos_test.jsonl  
  inflating: RuCoS/rucos_val.jsonl   
  inflating: __MACOSX/RuCoS/._rucos_val.jsonl  
  inflating: RuCoS/rucos_train.jsonl  
  inflating: __MACOSX/RuCoS/._rucos_train.jsonl  
mv: cannot move 'RuCoS/' to 'data/RuCoS': Directory not empty


In [20]:
!ls -lah data/

total 256M
drwxr-xr-x  3 jovyan jovyan 4.0K Mar  2 05:11 .
drwxrwxrwx 20 root   root   4.0K Mar  2 13:11 ..
drwxr-xr-x  2 jovyan jovyan 4.0K Mar  2 05:11 RuCoS
-rwxr-xr-x  1 jovyan jovyan 175M Oct 15 03:26 rucos_train.jsonl
-rw-r--r--  1 jovyan jovyan  17M Mar  2 05:09 test.jsonl
-rw-r--r--  1 jovyan jovyan  45M Mar  2 05:09 train.zip
-rw-r--r--  1 jovyan jovyan  19M Mar  2 05:09 val.jsonl


# Process data

In [21]:
class QADataset: 
    """Question and answers dataset""" 
     
    def __init__(self, path, making_smaller=False): 
        """ 
        Args: 
            path (string): Path to jsonl file 
        """ 
        self.path = path 
        data = [] 
        self.text = [] 
        with open(path, 'r') as json_file: 
            json_list = list(json_file) 
            for json_str in tqdm.tqdm(json_list): 
                item = json.loads(json_str) 
                text = item['passage']['text'] 
                self.text.append(text) 
                entities = item['passage']['entities'] 
                used = set()
                for en in entities:
                    word = text[en['start']:en['end']] 
                    if word not in used:
                        used.add(word)
                entities = list(used)

                for row in item['qas']: 
                    for ans in row.get('answers', [None]): 
                        q_text = row['query'] 
                        for i, word in enumerate(entities): 
                            n_q = q_text.replace('@placeholder', word) 
                            data.append({"idx": f'{item["idx"]}/{i}', 
                                         "text_id": len(self.text) - 1,
                                         "query": n_q})
                            
                            if not ans:
                                continue 
 
                            label = int(ans['text'].strip() == word.strip())
                            if not making_smaller or label or random.randint(1, 10) == 1:
                                data[-1]["label"] = label
                            else:
                                data.pop()
                self.text[-1] = re.sub(r'(@\w+)|(\\n)', " ", self.text[-1])
        self.data = pd.DataFrame.from_dict(data).set_index("idx")
     
    def __len__(self): 
        return len(self.data) 
 
    def __getitem__(self, ind):  
        if type(ind) == int: 
            row = self.data.iloc[ind] 
            text_a = self.text[row.text_id].split() 
            text_b = row.query 
 
            w = 512 - len(text_b.split()) - 5 
            e = random.randint(0, len(text_a) - w) 
            text_a = ' '.join(text_a[e:e + w]) 
 
            res = [[list(self.data.index)[ind], text_a, text_b, row.label if 'label' in self.data.columns else None]] 
            return pd.DataFrame(res, columns=["idx", "text_a", "text_b", "labels"]).set_index("idx") 
        if type(ind) == slice: 
            res = [] 
            rows = self.data.iloc[ind] 
            for idx, row in rows.iterrows(): 
                text_a = self.text[row.text_id].split() 
                text_b = row.query 
 
                w = 512 - len(text_b.split()) - 5 
                e = random.randint(0, max(1, len(text_a) - w)) 
                text_a = ' '.join(text_a[e:e + w]) 
 
                res.append([idx, text_a, text_b, row.label if 'label' in self.data.columns else None]) 
            return pd.DataFrame(res, columns=["idx", "text_a", "text_b", "labels"]).set_index("idx")
 
 
    def get_df(self): 
        texts = self.data['data'] 
        labels = self.data['label'] if 'label' in self.data.columns else [None] * self.data.shape[0] 
 
        return pd.DataFrame({ 
                    'text': texts, 
                    'labels': labels 
                })

In [22]:
train_df = QADataset('data/rucos_train.jsonl', making_smaller=True)
val_df = QADataset('data/val.jsonl')
test_df = QADataset('data/test.jsonl')

In [23]:
train_df[:].head()

,text_a,text_b,labels
idx,,,
0/3,"Наблюдатели полагают, что подоплекой теракта в...","Кроме того, серьезным вызовом для России стано...",1
1/1,О вторжении на Украину танковой колонны из РФ ...,Россия категорически опровергла сообщение Итал...,0
1/2,О вторжении на Украину танковой колонны из РФ ...,Россия категорически опровергла сообщение Киев...,0
1/8,О вторжении на Украину танковой колонны из РФ ...,Россия категорически опровергла сообщение СНБО...,0
1/9,О вторжении на Украину танковой колонны из РФ ...,Россия категорически опровергла сообщение Укра...,0


# Model params

In [24]:
# @title Model params { run: "auto" }
model_name = "DeepPavlov/rubert-base-cased" #@param {type:"string"}
num_train_epochs = 3 #@param {type:"slider", min:1, max:50, step:1}
use_cuda = True #@param ["True", "False"] {type:"raw"}
learning_rate =  4e-5 #@param {type:"number"}
train_batch_size = 32 #@param {type:"slider", min:1, max:100, step:1}
no_save = True #@param ["True", "False"] {type:"raw"}
adam_epsilon =  3e-8 #@param {type:"number"}

# Creating model

In [25]:
model = ClassificationModel(model_type='bert',
                            model_name=model_name,
                            use_cuda=use_cuda,
                            num_labels=2,
        
                            args=ClassificationArgs(
                                                    num_train_epochs=num_train_epochs,
                                                    learning_rate=learning_rate,
                                                    adam_epsilon=adam_epsilon,
                                                    
                                                    train_batch_size=train_batch_size,
                                                    max_seq_length=512,
                                                    manual_seed=RS,
                                                    
                                                    overwrite_output_dir=True,
                                                    no_cache=False,
                                                    no_save=no_save,
                                                    save_eval_checkpoints=False,
                                                    save_model_every_epoch=False,
                                                    save_steps=-1,
                                            )
                            )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training

In [26]:
model.train_model(train_df[:])

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(36000, 0.2672465087745546)

In [27]:
actual_test = test_df[:]

In [28]:
def save_pred(filename, preds):
    with open('data/test.jsonl', 'r') as json_file:
        test = list(map(lambda x: json.loads(x), list(json_file)))
    ind = 0
    res = []
    print(len(actual_test))
    print(len(test))
    
    for i in range(len(test)):
        ent_prob = []
        
        entities = test[i]['passage']['entities']
        used = set()
        for en in entities:
            word = test[i]['passage']['text'][en['start']:en['end']] 
            if word not in used:
                used.add(word)
        entities = list(used)
        
        for j in range(len(entities)):
            ent_prob.append(preds[ind])
            ind += 1
        ent_prob = np.array(ent_prob).argmax()
        
#         start, end = test[i]['passage']["entities"][ent_prob]['start'], test[i]['passage']["entities"][ent_prob]['end']
        answer = entities[ent_prob]
        res.append({"idx": test[i]["idx"], "text": answer})
    
    !mkdir submitions/
    
    with jsonlines.open("submitions/" + filename, 'w') as file:
        file.write_all(res)

In [31]:
predictions, raw_outputs = model.predict(list(zip(test_df[:].text_a, test_df[:].text_b)))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
cnt = Counter(predictions)

print(f'    cnt: {cnt}')
print(f"0 label: {cnt[0] / len(predictions) * 100:.3f}%")
print(f"1 label: {cnt[1] / len(predictions) * 100:.3f}%")

    cnt: Counter({0: 52365, 1: 15386})
0 label: 77.290%
1 label: 22.710%


In [33]:
from scipy.special import softmax
pred_proba = np.array([elem[1] for elem in softmax(raw_outputs, axis=1)])
pred_proba

array([7.62699662e-02, 9.96234586e-01, 2.04493945e-02, ...,
       2.48261100e-04, 8.49472369e-01, 2.50207749e-04])

In [34]:
file_name = f'{__import__("datetime").datetime.now()}.jsonl'
save_pred(file_name, pred_proba)

67751
7257
mkdir: cannot create directory ‘submitions/’: File exists


In [35]:
len(pred_proba)

67751